# **🚐 Lab 3**

Klasifikasi Naive Bayes dengan Data Multimoda.

### Langkah 1 - _Load_ Data

Pada tahap ini kita akan _loading_ data ke dalam data frame dan melakukan inspeksi sederhana untuk memastikan apakah kita perlu proses pra pengolahan data sebelum melakukan ekstraksi fitur dan permodelan.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import pandas as pd

In [ ]:
try:
    data = pd.read_csv('spam.csv', encoding='latin-1')
    print("File berhasil dibaca. Ini 5 baris pertamanya:")
    print(data.head())

except FileNotFoundError:
    print("Error: File 'iris.csv' tidak ditemukan.")
    
except pd.errors.EmptyDataError:
    print("Error: File 'iris.csv' ditemukan, tapi isinya kosong.")
    
except Exception as e:
    print(f"Terjadi error saat membaca file: {e}")

### Langkah 2 - Preprocessing

Beberapa hal yang akan dilakukan pada tahap ini yaitu,

- Drop kolom yang tidak digunakan

- Ubah nama kolom v1 (label) dan v2 (teks sms)

- Inspeksi Data

- Encode label

- Memisahkan fitur dengan label

### Langkah 2a - Drop Kolom

In [ ]:
# Drop 3 kolom terakhir dengan fungsi iloc
df = df.drop(df.iloc[:,2:], axis=1)

# Cek data
df.head()

### Langkah 2b - Inspeksi Data

In [ ]:
# Cek Jumlah Data Per Kelas
print(df['Labels'].value_counts())
print('\n')

# Cek Kelengkapan Data
print(df.info())
print('\n')

# Cek Statistik Deskriptif
print(df.describe())

### Langkah 2c - Encoding Label

In [ ]:
# Data untuk label
new_labels = {
    'spam': 1,
    'ham': 0
}

# Encode label
df['Labels'] = df['Labels'].map(new_labels)

# Cek data
df.head()

### Langkah 2d - Pisahkan Fitur dengan Label

In [ ]:
X = df['SMS'].values
y = df['Labels'].values

### Langkah 3 - Ekstraksi Fitur

Ekstraksi fitur untuk setiap SMS akan menggunakan konsep Bag of Words. Kita dapat menggunakan fungsi `CountVectorizer` dari scikit-learn. Akan tetapi untuk mencegah **leaking information** kita akan melakukan split data terlebih dahulu, baru melakukan transformasi terhadap data training dan testing.

In [ ]:
# Split data training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

# Inisiasi CountVectorizer
bow = CountVectorizer()

# Fitting dan transform X_train dengan CountVectorizer
X_train = bow.fit_transform(X_train)

# Transform X_test
# Mengapa hanya transform? Alasan yang sama dengan kasus pada percobaan ke-3
# Kita tidak menginginkan model mengetahui paramter yang digunakan oleh CountVectorizer untuk fitting data X_train
# Sehingga, data testing dapat tetap menjadi data yang asing bagi model nantinya
X_test = bow.transform(X_test)

Cek fitur dari proses `CountVectorizer`.

In [ ]:
print(len(bow.get_feature_names()))
print(f'Dimensi data: {X_train.shape}')

### Langkah 4 - Training dan Evaluasi Model

Kita akan menggunakan algoritma Multinomial Naive Bayes. Fungsi `MultinomialNB` dari scikit-learn dapat digunakan pada kasus ini.

In [ ]:
# Inisiasi MultinomialNB
mnb = MultinomialNB()

# Fit model
mnb.fit(X_train, y_train)

# Prediksi dengan data training
y_pred_train = mnb.predict(X_train)

# Evaluasi akurasi data training
acc_train = accuracy_score(y_train, y_pred_train)

# Prediksi dengan data training
y_pred_test = mnb.predict(X_test)

# Evaluasi akurasi data training
acc_test = accuracy_score(y_test, y_pred_test)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')py